In [3]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


import pandas as pd

# Task 1

In [5]:
df_x_train = pd.read_csv("uWaveGestureLibrary_X_TRAIN", delim_whitespace=True, header=None)
df_y_train= pd.read_csv("uWaveGestureLibrary_Y_TRAIN", delim_whitespace=True, header=None)
df_z_train = pd.read_csv("uWaveGestureLibrary_Z_TRAIN", delim_whitespace=True, header=None)

df_x_test = pd.read_csv("uWaveGestureLibrary_X_TEST", delim_whitespace=True, header=None)
df_y_test= pd.read_csv("uWaveGestureLibrary_Y_TEST", delim_whitespace=True, header=None)
df_z_test = pd.read_csv("uWaveGestureLibrary_Z_TEST", delim_whitespace=True, header=None)

In [6]:
df_x_train_velocity = df_x_train.loc[:, df_x_train.columns!=0].cumsum(axis = 1)
df_y_train_velocity = df_y_train.loc[:, df_y_train.columns!=0].cumsum(axis = 1)
df_z_train_velocity = df_z_train.loc[:, df_z_train.columns!=0].cumsum(axis = 1)

df_x_test_velocity = df_x_test.loc[:, df_x_test.columns!=0].cumsum(axis = 1)
df_y_test_velocity = df_y_test.loc[:, df_y_test.columns!=0].cumsum(axis = 1)
df_z_test_velocity = df_z_test.loc[:, df_z_test.columns!=0].cumsum(axis = 1)

In [7]:
df_x_train_position = df_x_train_velocity.cumsum(axis = 1)
df_y_train_position = df_y_train_velocity.cumsum(axis = 1)
df_z_train_position = df_z_train_velocity.cumsum(axis = 1)

df_x_test_position = df_x_test_velocity.cumsum(axis = 1)
df_y_test_position = df_y_test_velocity.cumsum(axis = 1)
df_z_test_position = df_z_test_velocity.cumsum(axis = 1)

In [8]:
concatenated_train = pd.concat([pd.concat([df_x_train_position, df_y_train_position], axis=1), df_z_train_position], axis = 1)
labels_train = np.array(df_x_train[0])
concatenated_test = pd.concat([pd.concat([df_x_test_position, df_y_test_position], axis=1), df_z_test_position], axis = 1)
labels_test = np.array(df_x_test[0])

# A

In [ ]:
# reference https://towardsdatascience.com/building-a-k-nearest-neighbors-k-nn-model-with-scikit-learn-51209555453a

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3)
# Fit the classifier to the data
knn.fit(concatenated_train,labels_train)
#train model with cv of 10
cv_scores = cross_val_score(knn, concatenated_train, labels_train, cv=10)


In [19]:
# euclidean distance
k_list = []
for d in range(2,10):
    # Create KNN classifier
    knn = KNeighborsClassifier(n_neighbors = d)
    # Fit the classifier to the data
    knn.fit(concatenated_train,labels_train)
    #train model with cv of 10
    cv_scores = cross_val_score(knn, concatenated_train, labels_train, cv=10)
    k_list.append(cv_scores.mean())

In [20]:
k_list

[0.8728089887640449,
 0.8739076154806492,
 0.8683645443196004,
 0.8739076154806492,
 0.8672159800249688,
 0.8672284644194758,
 0.8594132334581772,
 0.8615980024968788]

for euclidean distance best k is 5 

In [22]:
# chebyshev distance
k_list = []
for d in range(2,10):
    # Create KNN classifier
    knn = KNeighborsClassifier(n_neighbors = d, metric = 'chebyshev')
    # Fit the classifier to the data
    knn.fit(concatenated_train,labels_train)
    #train model with cv of 10
    cv_scores = cross_val_score(knn, concatenated_train, labels_train, cv=10)
    k_list.append(cv_scores.mean())


In [23]:
k_list

[0.85270911360799,
 0.8672784019975032,
 0.869450686641698,
 0.8716978776529338,
 0.8582646691635457,
 0.8594132334581772,
 0.8437453183520599,
 0.8393757802746566]

for chebyshev distance best k is 5 

# B

In [31]:
from sklearn.metrics import confusion_matrix
import time


start_time = time.time()

knn = KNeighborsClassifier(n_neighbors = 5)
# Fit the classifier to the data
knn.fit(concatenated_test,labels_test)

confusion_matrix(labels_test, knn.predict(concatenated_test))


array([[423,   0,   0,   3,   0,   6,   4,   1],
       [  2, 440,   0,   0,   0,   0,   8,   2],
       [  0,   2, 438,   0,   3,   7,   4,   0],
       [  1,   0,   1, 426,  19,   2,   0,   1],
       [  3,   0,  11,   2, 416,   0,   0,   1],
       [ 25,   0,   9,  14,  19, 380,   2,   0],
       [  5,  35,   3,   0,   0,   0, 397,   7],
       [  0,   5,   0,   2,   0,   0,   3, 450]])

In [32]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.4124269485473633 seconds ---


In [33]:
knn.score(concatenated_test, labels_test)

0.9408151870463428

In [34]:
start_time = time.time()


knn = KNeighborsClassifier(n_neighbors = 5, metric = 'chebyshev')
# Fit the classifier to the data
knn.fit(concatenated_test,labels_test)

confusion_matrix(labels_test, knn.predict(concatenated_test))



array([[426,   0,   0,   2,   0,   5,   3,   1],
       [  3, 439,   0,   0,   0,   0,   9,   1],
       [  0,   2, 438,   0,   3,   4,   6,   1],
       [  3,   0,   1, 423,  15,   5,   1,   2],
       [  3,   0,   9,   3, 415,   1,   0,   2],
       [ 27,   0,   7,  14,  21, 380,   0,   0],
       [  6,  39,   3,   0,   0,   0, 392,   7],
       [  0,   7,   0,   3,   0,   0,   1, 449]])

In [35]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 28.70748209953308 seconds ---


In [36]:
knn.score(concatenated_test, labels_test)

0.9385817978782803

# C 

I think that different weights in the distance from different axes makes real sense in terms of variability
difference in different axis. Meaning of distance variance in each axis represents the same idea of similarity
measure. For example, the movement and the acceleration in the x axis has more variance than in the z axis because of the nature of motion. The left handed and right handed gestures can be meaningful in terms of acceleration and position in the different axes. Therefore, different weights for each axes makes sense for classification of this motions.


# Task 2 

# A

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
concatenated_train

,1,2,3,4,5,6,7,8,9,10,...,306,307,308,309,310,311,312,313,314,315
0,-0.304243,-0.912730,-1.825459,-3.042432,-4.563648,-6.389107,-8.518810,-10.952756,-13.690944,-16.733377,...,-22393.513600,-22397.080506,-22400.143932,-22402.714390,-22404.810326,-22406.450185,-22407.652411,-22408.435450,-22408.817746,-22408.817745
1,1.627311,4.881933,9.763867,16.273111,24.409667,34.173533,45.564711,58.583200,73.229000,89.502111,...,21466.254875,21469.860898,21473.039910,21475.791750,21478.114817,21479.997165,21481.426845,21482.391912,21482.880419,21482.880418
2,0.661277,1.983830,3.967659,6.612765,9.919148,13.886807,18.515743,23.805955,29.757444,36.370209,...,23633.048145,23639.949883,23645.988903,23651.165206,23655.478792,23658.929660,23661.517812,23663.243246,23664.105963,23664.105963
3,0.005185,0.015554,0.031109,0.051848,0.077772,0.108880,0.145174,0.186652,0.233315,0.285163,...,5382.058335,5379.666500,5377.218796,5374.782899,5372.426489,5370.227135,5368.281520,5366.781408,5365.929835,5365.929835
4,1.286198,3.858593,7.717187,12.861978,19.292967,27.010154,36.013538,46.303121,57.878901,70.740879,...,-16912.338888,-16925.473284,-16936.851680,-16946.591494,-16954.810144,-16961.575528,-16966.825694,-16970.443222,-16972.310696,-16972.310698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,0.117811,0.353433,0.706866,1.178110,1.767165,2.474031,3.298708,4.241196,5.301495,6.479606,...,9342.102782,9341.457177,9340.745196,9340.022573,9339.326759,9338.695206,9338.165364,9337.774685,9337.560620,9337.560620
892,-1.232590,-3.697771,-7.395542,-12.325904,-18.488856,-25.884398,-34.512531,-44.373254,-55.466568,-67.792472,...,-15920.118999,-15922.438205,-15924.510225,-15926.322857,-15927.863896,-15929.121136,-15930.082374,-15930.735405,-15931.068023,-15931.068025
893,0.282877,0.848632,1.697264,2.828774,4.243161,5.940425,7.920567,10.183586,12.729483,15.558256,...,729.445893,728.678390,727.872724,727.067754,726.289916,725.551254,724.901513,724.410379,724.142552,724.142552
894,1.248704,3.746113,7.492226,12.487044,18.730566,26.222792,34.963723,44.953358,56.191698,68.678742,...,2145.998557,2149.444578,2152.700273,2155.659508,2158.211642,2160.259183,2161.747483,2162.686439,2163.119483,2163.119483


In [40]:
labels_train_2 = np.where(labels_train == 3, 1 , 0)

In [42]:
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()

In [43]:
logisticRegr.fit(concatenated_train, labels_train_2)

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [51]:
logisticRegr.predict_proba(concatenated_test)

array([[0.97544156, 0.02455844],
       [0.91116517, 0.08883483],
       [0.98777837, 0.01222163],
       ...,
       [0.06542714, 0.93457286],
       [0.92800068, 0.07199932],
       [0.98940781, 0.01059219]])

In [45]:
train_threshold = labels_train_2.sum()/ labels_train_2.size

In [46]:
train_threshold

0.11830357142857142

 # B

In [53]:
labels_test_2 = np.where(labels_test == 3, 1 , 0)

confusion_matrix(labels_test_2, np.where(logisticRegr.predict_proba(concatenated_test)[:,1] > train_threshold, 1, 0))



array([[1488, 1640],
       [   1,  453]])

In [54]:
from sklearn.linear_model import LogisticRegressionCV



In [62]:
# reference https://stackoverflow.com/questions/43581850/reproducing-lasso-logistic-regression-results-in-r-with-python-using-the-iris

clf = LogisticRegressionCV(penalty='l2', cv=10, scoring = "neg_log_loss", random_state=0)
clf.fit(concatenated_train, labels_train_2)

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mehmet.yildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

LogisticRegressionCV(cv=10, random_state=0, scoring='neg_log_loss')

In [63]:
clf.C_

array([10000.])

In [64]:
labels_test_2 = np.where(labels_test == 3, 1 , 0)

confusion_matrix(labels_test_2, np.where(clf.predict_proba(concatenated_test)[:,1] > train_threshold, 1, 0))




array([[1644, 1484],
       [   1,  453]])

In [65]:
clf.coef_

array([[-1.07807729e-07, -3.13723138e-07, -5.87532631e-07,
        -8.66891568e-07, -1.12571187e-06, -1.36463629e-06,
        -1.58401058e-06, -1.79043528e-06, -1.97489800e-06,
        -2.12110078e-06, -2.21497992e-06, -2.23521593e-06,
        -2.20116343e-06, -2.18681428e-06, -2.25456019e-06,
        -2.46298244e-06, -2.86549225e-06, -3.47334095e-06,
        -4.34559045e-06, -5.48322527e-06, -6.85520682e-06,
        -8.42958165e-06, -1.01673939e-05, -1.20519640e-05,
        -1.40923039e-05, -1.62348158e-05, -1.84021727e-05,
        -2.04775483e-05, -2.22723036e-05, -2.36360460e-05,
        -2.45117995e-05, -2.49196279e-05, -2.49669918e-05,
        -2.47005775e-05, -2.41783684e-05, -2.34971591e-05,
        -2.27258210e-05, -2.18768257e-05, -2.09790137e-05,
        -2.01125453e-05, -1.93437800e-05, -1.87015671e-05,
        -1.82238613e-05, -1.79182067e-05, -1.77158040e-05,
        -1.75753698e-05, -1.74820960e-05, -1.73452939e-05,
        -1.71947304e-05, -1.71465510e-05, -1.73597324e-0